<a href="https://colab.research.google.com/github/firarru/Text-Mining/blob/main/System_Recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_excel('https://raw.githubusercontent.com/firarru/Text-Mining/main/Dataset/data_hotel.xlsx')
df

,hotel_id,hotel_name,hotel_description,hotel_province,hotel_city,hotel_address,price_per_night
0,h0001,Midtown Residence Marvell City Surabaya,NaN,Jawa Timur,Surabaya,Jalan Ngagel Raya No 123,527866.666667
1,h0002,favehotel Graha Agung Surabaya,Sebuah Hotel Mewah di Surabaya Persembahan dar...,Jawa Timur,Surabaya,Jl. Mayjen Yono Soewoyo Pakuwon Indah Square A...,442860.000000
2,h0003,The Sun Hotel Sidoarjo,Hotel Bintang 3 Pertama dan Satu-satunya di Si...,Jawa Timur,Surabaya,"Jl. Pahlawan No.1,Sidokumpul, Sidoarjo, Suraba...",305000.000000
3,h0004,Grand Surabaya Hotel,Penginapan Yang Tenang Dan Nyaman Di Surabaya.,Jawa Timur,Surabaya,"Jl. Pemuda 19-21, Surabaya, Indonesia",324999.333333
4,h0005,The WIN Hotel Surabaya,WIN Hotel adalah hotel smart bintang 3 yang me...,Jawa Timur,Surabaya,"Jl. Embong Tanjung 46 - 48 Surabaya, Jawa Timu...",310947.250000
...,...,...,...,...,...,...,...
1194,h1195,Restu Hotel Yogyakarta,Akomodasi yang nyaman dan modern di Yogyakarta...,Daerah Istimewa Yogyakarta,Jogja,"Jl. Batikan No. 7, Pandeyan, Kec. Umbulharjo, ...",150000.000000
1195,h1196,Oryza Hotel Malioboro,Ideal accommodation options in the Malioboro a...,Daerah Istimewa Yogyakarta,Jogja,"Sosrowijayan St No.49-51, Sosromenduran, Gedon...",180000.000000
1196,h1197,Hotel Asri Graha Yogyakarta,"Pilihan akomodasi yang ideal untuk Pasangan, B...",Daerah Istimewa Yogyakarta,Jogja,"Jl. Veteran No. 184 A, Umbulharjo, Yogyakarta,...",262500.000000
1197,h1198,Tasneem Convention Hotel Yogyakarta,"Pilihan akomodasi yang ideal untuk Pasangan, B...",Daerah Istimewa Yogyakarta,Jogja,"Kawasan Mandira Baruga, Jl. Brigjen Katamso, K...",391666.666667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199 entries, 0 to 1198
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   hotel_id           1199 non-null   object 
 1   hotel_name         1199 non-null   object 
 2   hotel_description  1098 non-null   object 
 3   hotel_province     1199 non-null   object 
 4   hotel_city         1199 non-null   object 
 5   hotel_address      1199 non-null   object 
 6   price_per_night    1193 non-null   float64
dtypes: float64(1), object(6)
memory usage: 65.7+ KB


In [4]:
df_hotel = df[df['hotel_description'].notnull()]

In [5]:
pip install sastrawi

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import re
import random

In [7]:
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
sastrawi = StopWordRemoverFactory()
stopworda = sastrawi.get_stop_words()
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def clean_text(text):
    text = text.lower()
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = stemmer.stem(text)
    text = ' '.join(word for word in text.split() if word not in stopworda)
    return text

In [8]:
df_hotel['clean_description'] = df_hotel['hotel_description'].apply(clean_text)

<ipython-input-8-c574cc2f6328>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hotel['clean_description'] = df_hotel['hotel_description'].apply(clean_text)


In [9]:
df_hotel.head()

,hotel_id,hotel_name,hotel_description,hotel_province,hotel_city,hotel_address,price_per_night,clean_description
1,h0002,favehotel Graha Agung Surabaya,Sebuah Hotel Mewah di Surabaya Persembahan dar...,Jawa Timur,Surabaya,Jl. Mayjen Yono Soewoyo Pakuwon Indah Square A...,442860.000000,buah hotel mewah surabaya sembah archipelago i...
2,h0003,The Sun Hotel Sidoarjo,Hotel Bintang 3 Pertama dan Satu-satunya di Si...,Jawa Timur,Surabaya,"Jl. Pahlawan No.1,Sidokumpul, Sidoarjo, Suraba...",305000.000000,hotel bintang 3 pertama satusatunya sidoarjo
3,h0004,Grand Surabaya Hotel,Penginapan Yang Tenang Dan Nyaman Di Surabaya.,Jawa Timur,Surabaya,"Jl. Pemuda 19-21, Surabaya, Indonesia",324999.333333,inap tenang nyaman surabaya
4,h0005,The WIN Hotel Surabaya,WIN Hotel adalah hotel smart bintang 3 yang me...,Jawa Timur,Surabaya,"Jl. Embong Tanjung 46 - 48 Surabaya, Jawa Timu...",310947.250000,win hotel hotel smart bintang 3 tawar layan tu...
5,h0006,POP! Hotel Stasiun Kota Surabaya,Sebuah Hotel Modern yang Terjangkau di Jawa Timur,Jawa Timur,Surabaya,"Jl. Waspada No.58, Surabaya, Jawa Timur, Indon...",276333.333333,buah hotel modern jangkau jawa timur


In [10]:
hotel_df = df_hotel.copy()

In [11]:
hotel_df.set_index('hotel_name', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=1)
tfidf_matrix = tf.fit_transform(hotel_df['clean_description'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.00399128, 0.05252692, ..., 0.00584594, 0.00556788,
        0.0025404 ],
       [0.00399128, 1.        , 0.        , ..., 0.00263359, 0.00250833,
        0.00328081],
       [0.05252692, 0.        , 1.        , ..., 0.00533054, 0.005077  ,
        0.0536853 ],
       ...,
       [0.00584594, 0.00263359, 0.00533054, ..., 1.        , 0.6430531 ,
        0.01276909],
       [0.00556788, 0.00250833, 0.005077  , ..., 0.6430531 , 1.        ,
        0.01216174],
       [0.0025404 , 0.00328081, 0.0536853 , ..., 0.01276909, 0.01216174,
        1.        ]])

In [12]:
indices = pd.Series(hotel_df.index)
indices[:15]

,hotel_name
0,favehotel Graha Agung Surabaya
1,The Sun Hotel Sidoarjo
2,Grand Surabaya Hotel
3,The WIN Hotel Surabaya
4,POP! Hotel Stasiun Kota Surabaya
5,Evora Hotel Surabaya
6,MaxOneHotels at Dharmahusada
7,Zoom Dharmahusada Hotel
8,Neo+ Waru Sidoarjo by ASTON
9,favehotel Sidoarjo


In [13]:
# 5. Modelling recommender system
def recommendations(name, top = 10):

    recommended_hotel = []

    idx = indices[indices == name].index[0]
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    top = top + 1
    top_indexes = list(score_series.iloc[0:top].index)

    for i in top_indexes:
        recommended_hotel.append(list(hotel_df.index)[i]+" - "+str(score_series[i]))

    return recommended_hotel

In [14]:
recommendations('Gunawangsa Manyar Hotel', 15)

['Gunawangsa Manyar Hotel - 1.0000000000000002',
 'Midtown Hotel Surabaya - 0.17992312860891302',
 'Aria Gajayana Hotel - 0.14461427726403756',
 'GP Mega Kuningan Hotel - 0.123673904207206',
 'Le Polonia Medan managed by Topotels - 0.12049561758006844',
 'Swiss-Belinn Manyar Surabaya - 0.1173192927975267',
 'Hotel Santika Premiere Dyandra Hotel & Convention - 0.11442553936688896',
 'Best Western Plus Kemayoran Hotel - 0.1097598971613821',
 'Swiss-Belinn Malang - 0.09085670933358227',
 'Premier Place Surabaya Airport - 0.08833161676427399',
 'Quest Hotel Darmo - Surabaya by ASTON - 0.08833161676427399',
 'Zoom Hotel Jemursari - 0.08833161676427399',
 'Narita Classic Hotel Surabaya - 0.08833161676427399',
 'Cleo Hotel Basuki Rahmat - 0.07499752751020632',
 'Grand Darmo Suite by AMITHYA - 0.07174383261974006',
 'Varna Culture Hotel Surabaya - 0.06823993211613247']

Improving Content Based Filtering

In [35]:
df_review = pd.read_excel('https://raw.githubusercontent.com/firarru/Text-Mining/main/Dataset/review_hotel.xlsx')
df_review

,booking_id,booking_date,hotel_id,hotel_name,stay_duration,adults,children,rating,review
0,b0001,19-04-2020,h0014,Zest Hotel Jemursari Surabaya,1,2,1,8.4,Short stay
1,b0002,06-04-2020,h0014,Zest Hotel Jemursari Surabaya,1,1,1,10.0,Hotelnya nyaman
2,b0003,24-03-2020,h0014,Zest Hotel Jemursari Surabaya,2,2,1,9.2,Cukup baik untuk transit
3,b0004,23-03-2020,h0014,Zest Hotel Jemursari Surabaya,1,2,0,9.2,Nyaman
4,b0005,14-03-2020,h0014,Zest Hotel Jemursari Surabaya,2,2,1,6.8,Not good
...,...,...,...,...,...,...,...,...,...
4301,b4302,10-08-2019,h0791,The Luxton Bandung,1,2,2,3.6,Hotel kurang perawatan dan membosankan
4302,b4303,25-07-2019,h0791,The Luxton Bandung,2,2,2,6.8,Kurang perawatan
4303,b4304,08-07-2019,h0791,The Luxton Bandung,2,2,2,10.0,Mantul
4304,b4305,15-04-2019,h0791,The Luxton Bandung,3,2,2,3.6,Kecewa


In [36]:
df_review.isna().sum()

,0
booking_id,0
booking_date,0
hotel_id,473
hotel_name,0
stay_duration,0
adults,0
children,0
rating,0
review,0


In [37]:
# karena tanpa hotel_id maka data review tidak dapat direlasikan ke data hotel
df_review = df_review.dropna()

In [38]:
# 4. Group data review untuk setiap hotel
df_review = df_review.groupby('hotel_id', as_index=True).agg({'stay_duration':'mean', 'adults':'mean','children':'mean', 'rating':['mean','count']})

In [39]:
df_review.head()

stay_duration adults children rating      
                  mean   mean     mean   mean count
hotel_id                                           
h0001              1.4    1.6      0.4   9.20     5
h0002              1.3    1.3      0.5   8.32    10
h0003              1.6    1.6      0.4   8.76    10
h0004              1.3    1.5      0.5   8.52    10
h0005              1.4    1.6      0.3   8.04    10

In [41]:
# 5. Gabungkan data hotel dan review
hotel_review_df = hotel_df.merge(df_review, how='inner', on='hotel_id', right_index=True)

MergeError: Not allowed to merge between different levels. (1 levels on the left, 2 on the right)

In [ ]:
# 6. Rename colums hasil merge
hotel_review_df.rename(columns={hotel_review_df.columns[7]:'stay_duration', hotel_review_df.columns[8]:'adults',
                               hotel_review_df.columns[9]:'children', hotel_review_df.columns[10]:'rating',
                               hotel_review_df.columns[11]:'num_votes'}, inplace=True)

In [ ]:
# 7. Membagi kategori hotel berdasar harga
hotel_review_df['price_per_night'] = hotel_review_df['price_per_night'].astype(int)
hotel_review_df['price_bin'] = pd.cut(hotel_review_df['price_per_night'],
                                      bins=3, labels=['low','medium','high'])

In [ ]:
def set_family_room(x):
    if((x['children']>=1.0) | (x['adults']>1.0)):
        return "family room"
    else:
        return "single room"

In [ ]:
# 8. Set column family_room/no untuk setiap item hotel
hotel_review_df['preferences'] = hotel_review_df.apply(lambda x : set_family_room(x), axis=1)

In [ ]:
# 9. Mix column hotel_province, hotel_city, desc_clean, price_bin dan preferences sebagai metadata soup
hotel_review_df['metadata'] = hotel_review_df[['hotel_province','hotel_city','desc_clean','price_bin','preferences']].agg(' '.join, axis=1)

In [ ]:
# 10. Hitung TF-IDF dan Cosine similarity
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0,)
tfidf_matrix = tf.fit_transform(hotel_review_df['metadata'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim